In [3]:
from data_loading import load_tumors
import matplotlib.pyplot as plt
import glob
import pandas as pd
import numpy as np
from PIL import Image
from keras.layers import Input, Embedding, LSTM, Dense, Conv2D
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tqdm import tqdm

import tensorflow as tf
from tensorflow.python.client import device_lib
print device_lib.list_local_devices()

IMAGE_HEIGHT, IMAGE_WIDTH = 155, 240
TUMOR_IMAGES = 240
df = pd.read_csv("data/survival_data.csv")
df.head()

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4506144121430896262
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 368050176
locality {
  bus_id: 1
}
incarnation: 10955700225911154007
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0"
]


,Brats17ID,Age,Survival
0,Brats17_TCIA_167_1,74.907,153
1,Brats17_TCIA_242_1,66.479,147
2,Brats17_TCIA_319_1,64.860,254
3,Brats17_TCIA_469_1,63.899,519
4,Brats17_TCIA_218_1,57.345,346


In [4]:
IMAGES_PATH = '/mnt/naruto/Seminar-Med-Seg2017/BRATS2017/HGG/Png-HGG/'
X = np.zeros((len(df), IMAGE_HEIGHT, IMAGE_WIDTH, TUMOR_IMAGES))

for index, row in tqdm(df.iterrows(), total=len(df)):
    # For every patient, fetch his tumor pictures and sort them properly
    glob_path = "{}{}_{}*nx*.png".format(IMAGES_PATH, row['Brats17ID'], 'seg')
    tumor_images = glob.glob(glob_path)
    tumor_images.sort(key=lambda x : int(x.split('.')[-2]))
    
    # Convert every tumor image to greyscale and add it to X
    for i in range(len(tumor_images)):
        img = Image.open(tumor_images[i]).convert('LA')
        img = np.asarray(img, dtype=np.uint8)[:, :, 0]
        X[index, :, :, i] = img
X.shape

100%|██████████| 163/163 [04:34<00:00,  1.46s/it]


(163, 155, 240, 240)

In [5]:
np.save("tumors_nx", X)

### 163 patients with tumor images of sizes 155x240 of which we have 240

In [ ]:
Y = df['Survival']

In [ ]:
with tf.device('/gpu:0'):
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same', input_shape=X.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('relu'))

    # initiate RMSprop optimizer
    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

    # Let's train the model using RMSprop
    model.compile(loss='mean_squared_error', optimizer=opt)

    model.fit(X, Y)

Epoch 1/10
163/163 [==============================] - 123s - loss: 299581.5462   
Epoch 2/10
163/163 [==============================] - 18s - loss: 299581.5456    
Epoch 3/10
163/163 [==============================] - 18s - loss: 299581.5464    
Epoch 4/10
163/163 [==============================] - 18s - loss: 299581.5461    
Epoch 5/10
163/163 [==============================] - 18s - loss: 299581.5460    
Epoch 6/10
163/163 [==============================] - 18s - loss: 299581.5460    
Epoch 7/10
 32/163 [====>.........................] - ETA: 13s - loss: 350449.1562